In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow.keras import models, layers

2024-04-28 18:19:08.002929: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 18:19:08.002969: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 18:19:08.003013: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 18:19:08.011007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 18:19:08.749638: W tensorflow/compiler/

In [2]:
import os
from tensorflow.python.client import device_lib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(device_lib.list_local_devices() )

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13863278692558468062
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 38086639616
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7091141342492768754
physical_device_desc: "device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:81:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-04-28 18:19:09.805715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 18:19:09.944604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 18:19:09.944874: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
train_data = pd.read_csv('./dataset/emnist-balanced-train.csv')
test_data = pd.read_csv('./dataset/emnist-balanced-test.csv')

In [6]:
print(train_data.shape)
print(test_data.shape)

(112799, 785)
(18799, 785)


In [7]:
y_train = train_data.iloc[:,:1]
x_train = train_data.iloc[:,1:]

y_test = test_data.iloc[:,:1]
x_test = test_data.iloc[:,1:]

In [8]:
x_train_np = x_train.to_numpy()
x_test_np = x_test.to_numpy()

y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

In [9]:
x_train_np = np.reshape(x_train_np, newshape = (112799, 28, 28, 1))
x_test_np = np.reshape(x_test_np, newshape = (18799, 28, 28, 1))

y_train_np = tf.keras.utils.to_categorical(y_train_np,47)
y_test_np = tf.keras.utils.to_categorical(y_test_np,47)

In [10]:
model_lenet5 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, 
                    strides=1, activation='tanh', 
                    input_shape=[28,28,1], padding='same'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, 
                    strides=1, activation='tanh',padding='valid'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, 
                    strides=1, activation='tanh',padding='valid'), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(47, activation = "softmax")
])
model_lenet5.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
model_lenet5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (Average  (None, 14, 14, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 5, 5, 16)          0         
 gePooling2D)                                                    
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

2024-04-28 18:11:02.809490: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 18:11:02.809712: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [12]:
model_lenet5.fit(x_train_np, y_train_np, batch_size=32, epochs = 50, callbacks=[callbacks])

Epoch 1/50
 434/3525 [==>...........................] - ETA: 11s - loss: 1.4642 - accuracy: 0.5912

KeyboardInterrupt: 

In [ ]:
model_lenet5.evaluate(x_test_np,y_test_np)